## USAID 

The data is available in [USAID](https://explorer.usaid.gov/) site.

We download the data locally. 

We noticed that  the field `fiscal_year` has strange years `1976tq`, which denote some transition quarters. This will cause problem in our data analysis, so let us replace these years with proper year name: `1976`. To do this, we can resort to terminal command as follows. See [this](https://stackoverflow.com/questions/19456518/invalid-command-code-despite-escaping-periods-using-sed) for help. 

In [ ]:
!sed -i '' -e "s/1976tq/1976/g" us_foreign_aid_complete.csv

Next, I will read the file into a `DataFrame`. However, during loading pyhton returns some warning about two columns that has mixed datatypes. 

In [ ]:
import pandas as pd
import numpy as np

file_address = 'us_foreign_aid_complete.csv'
usaid_data = pd.read_csv(file_address)

Here are the various columns of our dataset

In [ ]:
usaid_data.columns

Upon investigating, we find that the columns `current_amount` and `constant_amount` indicate amounts as strings that has `,` separators for millions, thousands and etc, which the `pandas` converter `to_numeric` can not handle. 

To get around we must first remove these comma separators using `replace` function within list comprehension. However, this columns has already converted float which does not need any removal of comma's, hence we need to introduce conditional `if/else` within the list comprehension as follows.

In [ ]:
usaid_data['current_amount']= [x.replace(',', '') if isinstance(x, str) else x for x in usaid_data['current_amount'].values]
usaid_data['current_amount']= pd.to_numeric(usaid_data['current_amount'])

usaid_data['constant_amount']= [x.replace(',', '') if isinstance(x, str) else x for x in usaid_data['constant_amount'].values]
usaid_data['constant_amount']= pd.to_numeric(usaid_data['constant_amount'])

To make sure, let us display few lines from this column. 

In [ ]:
usaid_data['current_amount'].head()

What is the size of our dataset?

In [ ]:
usaid_data.shape

So, it has 821,744 records in 49 columns. Here is a list of the columns and their data types. 

In [ ]:
usaid_data.dtypes

In [ ]:
usaid_data['assistance_category_name'].unique()

If we think our dataset contains too many columns, we can delete unwanted column names like the ones containing ID's, codes or acronyms. 

In [ ]:
del_list = [c for c in usaid_data.columns.str.contains(r'_id|_code|_acronym')]
import itertools
drop_list =[c for c in itertools.compress(usaid_data.columns.values, del_list)]
usaid_data.drop(drop_list,1)

What period this dataset covers?

In [ ]:
print(usaid_data['numeric_year'].min())
print(usaid_data['numeric_year'].max())

So, we have 71 years of US Aid data starting from after World War II till today. 

Let us see to how many countries did USA extended its aid throughut the last 71 years.

In [ ]:
len(usaid_data['country_name'].unique())

Here is the list of countries. Note that this list contains some regions as well like 'Middle East'. It is good to note that region names end up with the word `Region`, so we can grab them all if we want a specific regional analysis.

In [ ]:
countries = usaid_data['country_name'].unique().tolist()
print(sorted(countries))

Let us see how much a particular country gets US Aid in a particular year. I have placed this inside a function that takes a country name and year (defaulted to 2015) and transaction type (defaulted to 'Obligation') and returned the sum of all aides in that year. Note, that I have used the `format_currency` module from `babel.numbers` to return a better formatted number.

In [ ]:
def country_aid(country, year=2015, type='Obligations', format='c'):
    mask_country = usaid_data['country_name']== country
    mask_transaction = usaid_data['transaction_type_name']==type
    mask_year = usaid_data['fiscal_year']==year
    res = usaid_data.loc[mask_country & mask_transaction & mask_year]
    result = res.loc[:,'current_amount'].sum()
    if format=='c':
        from babel.numbers import format_currency
        return(format_currency(result, 'USD', locale='en_US'))
    else:
        return result

In [ ]:
country_aid('Iran', 2015, format='c')

Let us leverage on this function to create a list of aid amounts for a range of years.

In [ ]:
[country_aid('Bangladesh', x, format='n') for x in range(2001,2015)]

Let us put togather these snippets to create a function that plots US Aid to a given country for a given time period. 

In [ ]:
def plot_trend(country, start=2001, end=2015):
    %matplotlib inline
    import matplotlib.pyplot as plt
    amts = [country_aid(country, x, format='n') for x in range(start,end+1)]
    fig, axis = plt.subplots()
    title = "USAid to %s [%d - %d]" %(country, start, end)
    plt.title(title)
    plt.ylabel('Current US$')
    axis.plot(range(start,end+1), amts)

In [ ]:
plot_trend('Bangladesh')

In [ ]:
plot_trend('Saudi Arabia', 1946)

In [ ]:
plot_trend("Bangladesh", 1970, 1973)

`assistance_category_name` specifies military or economic classification of the aid. Let us tweak the previous function to find out what % of USAid to a country goes for military puposes. 

In [ ]:
def aid_category(country, year_start=1946, year_end=2015, type='Obligations'):
    mask_country = usaid_data['country_name']== country
    mask_transaction = usaid_data['transaction_type_name']==type
    mask_year_start = usaid_data['fiscal_year']>=year_start
    mask_year_end = usaid_data['fiscal_year']<=year_end
    res = usaid_data.loc[mask_country & mask_transaction & mask_year_start & mask_year_end]
    total = res.loc[:,'current_amount'].sum()
    res2 = res.groupby('assistance_category_name').agg('sum')
    try:
        return res2.loc[:, 'current_amount'][1]/total * 100
    except:
        return 0

In [ ]:
aid_category('Bangladesh')

Using this we can build a `dictionary` of all countries and the proportaion of military aid from USA. 

In [ ]:
countries = usaid_data['country_name'].unique().tolist()
dic = {}
for c in countries:
    dic[c]= aid_category(c)


To find the top countries, all we need is to find a way to sort this dictionary by values.

In [ ]:
sorted(dic.items(), key=lambda x: x[1], reverse=True)

How about finding the top countries by military aid values (rather than proportion of total aids).

In [ ]:
def military_aid(country, year_start=1946, year_end=2015, type='Obligations'):
    mask_country = usaid_data['country_name']== country
    mask_transaction = usaid_data['transaction_type_name']==type
    mask_year_start = usaid_data['fiscal_year']>=year_start
    mask_year_end = usaid_data['fiscal_year']<=year_end
    mask_aid_category = usaid_data['assistance_category_name']=='Military'
    res = usaid_data.loc[mask_country & mask_transaction & mask_year_start & mask_year_end & mask_aid_category]
    total = res.loc[:,'current_amount'].sum()
    return total

In [ ]:
military_aid('Bangladesh')

In [ ]:
countries = usaid_data['country_name'].unique().tolist()
dic2 = {}
for c in countries:
    dic2[c]= military_aid(c)

In [ ]:
mil = sorted(dic2.items(), key=lambda x: x[1], reverse=True)
mil

No surprizing that Israel is the champion of US Military aid with the figure of 88.5 billion US Dollar. 

Let us vizualize the top 10 countries through a horizontal bar chart. To do this, we first need to prepare the top 10 country names from the first element of the tuple. 

In [ ]:
top10countries = [x for x,y in mil][:10]
amts = [y for x,y in mil][:10]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcdefaults()
fig, ax = plt.subplots()

y_pos = np.arange(len(top10countries))
error = np.random.rand(len(top10countries))

ax.barh(y_pos, amts, xerr = error, 
        align='center',
#        color='red',
        ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(cc)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('US$ Aid Amounts')
ax.set_title('Top 10 US Military Aid Receipients')

plt.show()

We want to draw trend line for a country showing the progress of US Aid for both economic and military category. This show tell us US interest about a country over time and how it favors militarism to a country.

In [ ]:
def country_aid_cat(country, cat='Economic', year=2015, type='Obligations', format='c'):
    mask_country = usaid_data['country_name']== country
    mask_transaction = usaid_data['transaction_type_name']==type
    mask_year = usaid_data['fiscal_year']==year
    mask_cat = usaid_data['assistance_category_name']==cat
    res = usaid_data.loc[mask_country & mask_transaction & mask_year & mask_cat]
    result = res.loc[:,'current_amount'].sum()
    if format=='c':
        from babel.numbers import format_currency
        return(format_currency(result, 'USD', locale='en_US'))
    else:
        return result

In [ ]:
country_aid_cat('Bangladesh', cat='Military')

In [ ]:
usaid_data['assistance_category_name'].unique()

In [ ]:
[country_aid_cat('Bangladesh', 'Military',x, format='n') for x in range(2001,2015)]

In [ ]:
def plot_trend_cat(country, start=2001, end=2015):
    %matplotlib inline
    import matplotlib.pyplot as plt
    amts_econ = [country_aid_cat(country,'Economic', x, format='n') for x in range(start,end+1)]
    amts_mil = [country_aid_cat(country,'Military', x, format='n') for x in range(start,end+1)]
    fig, axis = plt.subplots()
    title = "USAid to %s [%d - %d]" %(country, start, end)
    plt.title(title)
    plt.ylabel('Current US$')
    axis.plot(range(start,end+1), amts_econ)
    axis.plot(range(start,end+1), amts_mil)
    plt.legend(['Economic', 'Mililary'])
    plt.show()

In [ ]:
plot_trend_cat('Vietnam (former South)', 1960,1980)

In [ ]:
usaid_data.columns[43:45]

In [ ]:
usaid_data.dtypes

In [ ]:
print(usaid_data['fiscal_year'].min())

In [ ]:
plot_trend_cat('Iran', 1946)

In [ ]:
plot_trend_cat('Iran', 1950,1960)

In [ ]:
plot_trend_cat('Iran', 1960, 1971)

In [ ]:
def country_aid_activities(country, year=2015, type='Obligations', format='c'):
    mask_country = usaid_data['country_name']== country
    mask_transaction = usaid_data['transaction_type_name']==type
    mask_year = usaid_data['fiscal_year']==year
    res = usaid_data.loc[mask_country & mask_transaction & mask_year]
    res = res.groupby('activity_name').agg('sum')
    res = res.filter(['activity_name', 'current_amount'], axis=1)
    return res.sort_values('current_amount', ascending=False)[:10]

In [ ]:
country_aid_activities('Bangladesh')

In [ ]:
country_aid_activities('Iran', 1956)

In [ ]:
def average_aids(country, year_start=1946, year_end=2015, type='Obligations'):
    mask_country = usaid_data['country_name']== country
    mask_transaction = usaid_data['transaction_type_name']==type
    mask_year_start = usaid_data['fiscal_year']>=year_start
    mask_year_end = usaid_data['fiscal_year']<=year_end
    res = usaid_data.loc[mask_country & mask_transaction & mask_year_start & mask_year_end]
    res = res.groupby('fiscal_year').agg('mean')
    res = res.filter(['fiscal_year', 'current_amount'], axis=1)
    return res

In [ ]:
average_aids('Iran')

In [ ]:
plot_trend_cat('Israel', 1970, 2015)

In [ ]:
plot_trend_cat('West Bank/Gaza', 1970, 2015)

In [ ]:
country_aid_activities('Israel')

In [ ]:
country_aid_activities('West Bank/Gaza', 2016)

In [ ]:
plot_trend_cat('Israel', 1970, 1981)

In [ ]:
plot_trend_cat('Egypt', 1975)

In [ ]:
plot_trend_cat('Jordan', 1946)

In [ ]:
tot_all = usaid_data['current_amount'].sum()

In [ ]:
israel = usaid_data['country_name']== 'Israel'
egypt = usaid_data['country_name']== 'Egypt'
il = usaid_data.loc[israel]
tot_il = il['current_amount'].sum()
eg = usaid_data.loc[egypt]
tot_eg = eg['current_amount'].sum()

In [ ]:
tot_eg

In [ ]:
tot_il

In [ ]:
(tot_eg + tot_il)/tot_all